In [25]:
from datetime import datetime
from dateutil.parser import parse
from langchain_core.tools import Tool
from langchain.agents import initialize_agent, AgentType
from langchain_openai import ChatOpenAI
from langchain.tools import tool

In [26]:
# 讀取環境變數(OpenAI API Key)
from dotenv import load_dotenv
load_dotenv()

True

In [27]:
# Tool 1
@tool
def get_current_time() -> str:
    """取得目前時間，無需輸入參數"""
    now = datetime.now()
    return now.strftime("%Y-%m-%d %H:%M:%S")

In [28]:
# Tool 2
@tool
def calculate_time_difference(start_time: str, end_time: str) -> str:
    """
    計算兩個時間之間已過或剩餘多久。
    start_time: 開始時間，格式為 'YYYY-MM-DD HH:MM:SS'
    end_time: 結束時間，格式為 'YYYY-MM-DD HH:MM:SS'
    """
    try:
        start = parse(start_time)
        end = parse(end_time)
        if start >= end:
            delta = start - end
            prefix = "已過"
        else:
            delta = end - start
            prefix = "還有"
        days = delta.days
        hours = delta.seconds // 3600
        minutes = (delta.seconds % 3600) // 60
        return f"{prefix} {days} 天 {hours} 小時 {minutes} 分鐘"
    except Exception as e:
        return f"錯誤: {e}"

In [29]:
# 組成工具列表
tools = [get_current_time, calculate_time_difference]

In [30]:
# 使用 gpt-3.5-turbo
llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0
)

In [31]:
# 使用 Function Calling 模式
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.OPENAI_FUNCTIONS,
    verbose=True
)

In [ ]:
# 測試輸入
query = "現在幾點?距離 2025-08-01 08:00:00 還有多久?"
# query = "2025-08-02 10:00 距離 2025-08-01 08:00:00 還有多久?"

# 執行呼叫
response = agent.invoke({"input": query})

print(response["output"])



> Entering new AgentExecutor chain...

Invoking: `calculate_time_difference` with `{'start_time': '2025-08-01 08:00:00', 'end_time': '2025-08-02 10:00'}`


還有 1 天 2 小時 0 分鐘距離 2025-08-02 10:00 還有 1 天 2 小時 0 分鐘。

> Finished chain.
距離 2025-08-02 10:00 還有 1 天 2 小時 0 分鐘。
